# Imports

In [1]:
import os
import sys
import math
import time
import argparse
import numpy as np

from PIL import Image

from datetime import datetime

import dataGenerator
import logly

import torch
import torch.utils
import torch.utils.data
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler

import torchvision
import torchvision.utils as vutils
from torchvision.transforms import ToPILImage
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms

from livelossplot import PlotLosses

# User Settings

In [2]:
import pickle

# class Company(object):
#     def __init__(self, name, value):
#         self.name = name
#         self.value = value

# with open('company_data.pkl', 'wb') as output:
#     company1 = Company('banana', 40)
#     pickle.dump(company1, output, pickle.HIGHEST_PROTOCOL)

#     company2 = Company('spam', 42)
#     pickle.dump(company2, output, pickle.HIGHEST_PROTOCOL)

# del company1
# del company2

# with open('company_data.pkl', 'rb') as input:
#     company1 = pickle.load(input)
#     print(company1.name)  # -> banana
#     print(company1.value)  # -> 40

#     company2 = pickle.load(input)
#     print(company2.name) # -> spam
#     print(company2.value)  # -> 42

    
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

# sample usage
#save_object(company1, 'company1.pkl')

# Define DataLoader

In [2]:
#Set Values
show_live_loss = False
newTraining = True
save_masks = False
print_loss = True
output_tensors = False
train_flag = True
EPOCHS = 3
iteration = "-10"

# default values
NUM_CLASSES = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batchSize = 100
#val_batch_size = 100
num_batch = 1000 #25000 #11112
val_batch = 1
imageSize = 416
SAVE_LOCATION = "./data/models/model_test"
LOAD_LOCATION = "./data/models/model_best"
INTERRUPTED_LOCATION =  "./data/models/model_interrupted"
train_dir = "./data/train/"
val_dir = "./data/validation/"
log_dir = './data/logs/'

from torchvision import datasets

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """
    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        
        # the image file path
        path = self.imgs[index][0]
        
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        
        return tuple_with_path

# Load Data

In [3]:
data_transforms = transforms.Compose([transforms.Resize([imageSize,imageSize]),
                                      transforms.ToTensor()
                                     ])

# instantiate the dataset and dataloader
train_dataset = ImageFolderWithPaths(train_dir, transform=data_transforms) # our custom dataset
val_dataset = ImageFolderWithPaths(val_dir, transform=data_transforms) # our custom dataset

#loads only photos
train_dataloaders = torch.utils.data.DataLoader(train_dataset, batch_size = batchSize, shuffle=True)
val_dataloaders = torch.utils.data.DataLoader(val_dataset, batch_size = batchSize, shuffle=True)

data_factory = dataGenerator.dataGenerator(IMAGE_SIZE=imageSize)

# Training W/O Val Steps

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=4):
    since = time.time()
    best_model = None
    best_loss = math.inf
    #model.train()  
    #liveloss = PlotLosses()
    model = model.to(device)
    sigmoid_function = nn.Sigmoid()
    batch_log = logly.logly(True, num_epochs, num_batch)
    epoch_log = logly.logly(True, 1, num_epochs)
    
    for epoch in range(num_epochs):
        if print_loss:
            print('Epoch {}/{}'.format(epoch + 1, num_epochs), flush=True)
        
        #Training    
        model.train()
        phase = 'train'
        epoch_loss = 0
        for batch in range(num_batch):
            running_loss = 0.0
            
            if print_loss:
                #print('Epoch {}/{}'.format(epoch + 1, num_epochs), flush=True)
                print('Batch {}/{}'.format(batch + 1, num_batch), flush=True)
            
            #BATCH TUPLE
            inputs, labels, paths = next(iter(train_dataloaders))
            inputs = inputs.to(device)
            #labels.to(device)

            # Moved outside paths forloop as labels would be reset
            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                #dtype=torch.int64
                ### Why are we using float?
                #labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize, dtype = torch.float32).to(device)
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                #print("loading tensor:", i,  loc, flush=True)
                #print("tensor max:", torch.max(labels[i]))
                i += 1

            if train_flag:
                phase = 'train'
                #print("Setting model in training mode")
                # Set model to training mode
                model.train() 
            else:
                phase = 'val'
                #print("Setting model in validation mode")
                # Set model to evaluate mode
                model.eval()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                #outputs = model(inputs.to(device)).to(device)
                #outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                if phase == 'train':
                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # backward + optimize 
                    loss.backward() #modifies model
                    optimizer.step() 

                # statistics
                epoch_loss += loss.item()
                running_loss = loss.item()

            #
            
            batch_log.log(phase, epoch, batch, running_loss)
            
            if batch % 20 == 0:
                log_name = str(datetime.now().date()) + iteration
                batch_log.write(log_dir, log_name)
            
            if print_loss:
                print("loss: {}".format(running_loss), flush=True)
                print('---------------', flush=True)
        
        epoch_loss /= val_batch
        epoch_log.log(phase, 0, epoch, epoch_loss)
            
        # Validation
        model.eval()
        phase = 'val'
        val_loss = 0
        for batch in range(val_batch):
            if print_loss:
                print('Val-Batch {}/{}'.format(batch + 1, val_batch), flush=True)
            
            #BATCH TUPLE
            inputs, labels, paths = next(iter(train_dataloaders))
            #inputs, labels, paths = next(iter(val_dataloaders))
            inputs = inputs.to(device)
            #labels.to(device)

            # Moved outside paths forloop as labels would be reset
            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                i += 1
            
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                # statistics
                val_loss += loss.item()
                running_loss = loss.item()

            batch_log.log(phase, epoch, batch, running_loss)
            
            if batch % 20 == 0:
                log_name = str(datetime.now().date()) + iteration
                batch_log.write(log_dir, log_name)
                
            if print_loss:
                print("loss: {}".format(running_loss), flush=True)
                print('---------------', flush=True)
            
            if save_masks and phase == 'val':
                outputs = sigmoid_function(outputs)
                # need to fix before running
                outputs_dir = "data/outputs/results/" + str(datetime.now().date())
                os.makedirs(outputs_dir, exist_ok=True)
                threshold = 0.05
                j = 0
                for locations in paths:
                    img = Image.open(locations)
                    num = locations.replace("./data/train/photos/", "").replace(".png", "")
#                     num = locations.replace("./data/validation/photos/", "").replace(".png", "")
                    for threshold in [0.05, .25, .5, .75]:
                        img = data_factory.showMaskOnImage(img,outputs[j], threshold)
                        img.save(outputs_dir  + "/train-" + str(num) + "-" + str(epoch) + "-t" + str(threshold) + ".png","PNG")
                    j += 1
                
        val_loss /= num_batch
        epoch_log.log(phase, 0, epoch, val_loss)
        
        epoch_log.write(log_dir, "epoch - "+ str(datetime.now().date()) + iteration)    

#             if show_live_loss and phase == 'train':
#                 logs['log loss'] = epoch_loss
#                 logs['accuracy'] = 1 - epoch_loss
#                 liveloss.update(logs)
#                 liveloss.draw()

            

            
                #input("please enter any key to continue")

#             if output_tensors:
#                 return inputs, outputs, paths

        #save best copy of  model
        #if phase == 'val' and epoch_loss < best_loss:
#         if epoch_loss < best_loss:
#             best_loss = epoch_loss
#             torch.save(model, LOAD_LOCATION)
#             torch.save(model, SAVE_LOCATION)
#             torch.save(model.state_dict(), LOAD_LOCATION + "-" + str(datetime.now().date()) + iteration) 
#             #torch.save(model, SAVE_LOCATION)
#             #torch.save(model, SAVE_LOCATION.replace("model", "model_best"))
        
    time_elapsed = time.time() - since
    
    print('Training complete in {:.1f}m {:.1f}s'.format(time_elapsed // 60, time_elapsed % 60), flush=True)
    
    log_name = str(datetime.now().date()) + iteration
    batch_log.write(log_dir, log_name)
    
    epoch_log.write(log_dir, "epoch - "+ str(datetime.now().date()) + iteration)
    
    #logs_dir = "data/logs/" + str(datetime.now().date())
    #save_object(log, logs_dir)
    
    #completed model
    #torch.save(model,SAVE_LOCATION)
    return model

# Load Pretrained Model Weights

In [5]:
from unet_models import *

#imports related to UNet
if newTraining:
    model = UNet1(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
    print('Single convolution model with weights', flush=True)
#     model = UNet16_5(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
    
#     print('initializing model with random weights', flush=True)
#     torch.nn.init.xavier_uniform_(next(model.center.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec5.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec4.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec3.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec2.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec1.children()).weight)
#     torch.nn.init.xavier_uniform_(model.final.weight)
                           
else:
    print("loading weights from", LOAD_LOCATION, flush=True)
    #model = torch.load(SAVE_LOCATION)
    model = torch.load(LOAD_LOCATION)
    #model.load_state_dict(best_model_wts)

criterion = torch.nn.BCEWithLogitsLoss()
#criterion = DICELossMultiClass()
#criterion = IOU_BCELoss()

#Observe adjustments in learning rate
optimizer_ft = optim.Adam(model.parameters(), lr=0.05, weight_decay=0, amsgrad=False, eps=0.1)

# Osscilate between high and low learning rates over time
exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=EPOCHS,eta_min=0.001)  


#model = model.to(device)

Single convolution model with weights?


# Training and Results

In [6]:
try:
    model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

Epoch 1/3
Batch 1/1000
loss: 0.698851466178894
---------------
Batch 2/1000
loss: 0.7001091241836548
---------------
Batch 3/1000
loss: 0.6977899670600891
---------------
Batch 4/1000
loss: 0.697952926158905
---------------
Batch 5/1000
loss: 0.697689414024353
---------------
Batch 6/1000
loss: 0.6972122192382812
---------------
Batch 7/1000
loss: 0.6969905495643616
---------------
Batch 8/1000
loss: 0.697983980178833
---------------
Batch 9/1000
loss: 0.6964943408966064
---------------
Batch 10/1000
loss: 0.6962066888809204
---------------
Batch 11/1000
loss: 0.6948216557502747
---------------
Batch 12/1000
loss: 0.695912778377533
---------------
Batch 13/1000
loss: 0.6972399950027466
---------------
Batch 14/1000
loss: 0.6963637471199036
---------------
Batch 15/1000
loss: 0.6961931586265564
---------------
Batch 16/1000
loss: 0.6952420473098755
---------------
Batch 17/1000
loss: 0.6962416768074036
---------------
Batch 18/1000
loss: 0.6959665417671204
---------------
Batch 19/1000


---------------
Batch 150/1000
loss: 0.6924890279769897
---------------
Batch 151/1000
loss: 0.6891195774078369
---------------
Batch 152/1000
loss: 0.6894670128822327
---------------
Batch 153/1000
loss: 0.6911498308181763
---------------
Batch 154/1000
loss: 0.6897518634796143
---------------
Batch 155/1000
loss: 0.6925819516181946
---------------
Batch 156/1000
loss: 0.6932936906814575
---------------
Batch 157/1000
loss: 0.6906583905220032
---------------
Batch 158/1000
loss: 0.6905469298362732
---------------
Batch 159/1000
loss: 0.6910042762756348
---------------
Batch 160/1000
loss: 0.6900613307952881
---------------
Batch 161/1000
loss: 0.692179799079895
---------------
Batch 162/1000
loss: 0.6919687390327454
---------------
Batch 163/1000
loss: 0.6923952102661133
---------------
Batch 164/1000
loss: 0.6902596950531006
---------------
Batch 165/1000
loss: 0.6923447251319885
---------------
Batch 166/1000
loss: 0.691097617149353
---------------
Batch 167/1000
loss: 0.69053483009

---------------
Batch 297/1000
loss: 0.6898353695869446
---------------
Batch 298/1000
loss: 0.6912612318992615
---------------
Batch 299/1000
loss: 0.6912198066711426
---------------
Batch 300/1000
loss: 0.6923689246177673
---------------
Batch 301/1000
loss: 0.6911102533340454
---------------
Batch 302/1000
loss: 0.6913731694221497
---------------
Batch 303/1000
loss: 0.6909405589103699
---------------
Batch 304/1000
loss: 0.6902563571929932
---------------
Batch 305/1000
loss: 0.6905344128608704
---------------
Batch 306/1000
loss: 0.6904845833778381
---------------
Batch 307/1000
loss: 0.6921221613883972
---------------
Batch 308/1000
loss: 0.691189169883728
---------------
Batch 309/1000
loss: 0.6918116807937622
---------------
Batch 310/1000
loss: 0.6906782984733582
---------------
Batch 311/1000
loss: 0.689909040927887
---------------
Batch 312/1000
loss: 0.6925187110900879
---------------
Batch 313/1000
loss: 0.6910365223884583
---------------
Batch 314/1000
loss: 0.69108694791

loss: 0.6902946829795837
---------------
Batch 444/1000
loss: 0.6921418309211731
---------------
Batch 445/1000
loss: 0.6910266280174255
---------------
Batch 446/1000
loss: 0.6920570135116577
---------------
Batch 447/1000
loss: 0.6905838847160339
---------------
Batch 448/1000
loss: 0.6907426714897156
---------------
Batch 449/1000
loss: 0.6916365623474121
---------------
Batch 450/1000
loss: 0.6912615299224854
---------------
Batch 451/1000
loss: 0.690941333770752
---------------
Batch 452/1000
loss: 0.6905616521835327
---------------
Batch 453/1000
loss: 0.6921772360801697
---------------
Batch 454/1000
loss: 0.690518319606781
---------------
Batch 455/1000
loss: 0.6910578608512878
---------------
Batch 456/1000
loss: 0.6904626488685608
---------------
Batch 457/1000
loss: 0.6904402375221252
---------------
Batch 458/1000
loss: 0.6921648383140564
---------------
Batch 459/1000
loss: 0.6910077929496765
---------------
Batch 460/1000
loss: 0.6900695562362671
---------------
Batch 461

Batch 590/1000
loss: 0.6895240545272827
---------------
Batch 591/1000
loss: 0.6916602253913879
---------------
Batch 592/1000
loss: 0.6913968920707703
---------------
Batch 593/1000
loss: 0.6916464567184448
---------------
Batch 594/1000
loss: 0.6909217238426208
---------------
Batch 595/1000
loss: 0.6906691789627075
---------------
Batch 596/1000
loss: 0.6910156011581421
---------------
Batch 597/1000
loss: 0.6918026804924011
---------------
Batch 598/1000
loss: 0.6909220218658447
---------------
Batch 599/1000
loss: 0.6915258765220642
---------------
Batch 600/1000
loss: 0.6911996006965637
---------------
Batch 601/1000
loss: 0.6922088861465454
---------------
Batch 602/1000
loss: 0.6914449334144592
---------------
Batch 603/1000
loss: 0.6919354200363159
---------------
Batch 604/1000
loss: 0.6895592212677002
---------------
Batch 605/1000
loss: 0.6913465261459351
---------------
Batch 606/1000
loss: 0.6902463436126709
---------------
Batch 607/1000
loss: 0.691838800907135
---------

...

Batch 353/1000
loss: 0.6901801824569702
---------------
Batch 354/1000
loss: 0.6909554600715637
---------------
Batch 355/1000
loss: 0.6909604072570801
---------------
Batch 356/1000
loss: 0.6899541020393372
---------------
Batch 357/1000
loss: 0.6917637586593628
---------------
Batch 358/1000
loss: 0.6918299198150635
---------------
Batch 359/1000
loss: 0.6908565759658813
---------------
Batch 360/1000
loss: 0.6910598874092102
---------------
Batch 361/1000
loss: 0.6908960342407227
---------------
Batch 362/1000
loss: 0.6920867562294006
---------------
Batch 363/1000
loss: 0.6910126209259033
---------------
Batch 364/1000
loss: 0.6909725666046143
---------------
Batch 365/1000
loss: 0.689021646976471
---------------
Batch 366/1000
loss: 0.6913584470748901
---------------
Batch 367/1000
loss: 0.6895193457603455
---------------
Batch 368/1000
loss: 0.6915979981422424
---------------
Batch 369/1000
loss: 0.6903465390205383
---------------
Batch 370/1000
loss: 0.6900562047958374
---------

---------------
Batch 500/1000
loss: 0.6903768181800842
---------------
Batch 501/1000
loss: 0.6906970143318176
---------------
Batch 502/1000
loss: 0.6941406726837158
---------------
Batch 503/1000
loss: 0.6906366348266602
---------------
Batch 504/1000
loss: 0.6918384432792664
---------------
Batch 505/1000
loss: 0.6904986500740051
---------------
Batch 506/1000
loss: 0.6907539963722229
---------------
Batch 507/1000
loss: 0.6916770339012146
---------------
Batch 508/1000
loss: 0.690819501876831
---------------
Batch 509/1000
loss: 0.6914553642272949
---------------
Batch 510/1000
loss: 0.6909288763999939
---------------
Batch 511/1000
loss: 0.6915220022201538
---------------
Batch 512/1000
loss: 0.6913840770721436
---------------
Batch 513/1000
loss: 0.691595196723938
---------------
Batch 514/1000
loss: 0.6901708245277405
---------------
Batch 515/1000
loss: 0.6912059783935547
---------------
Batch 516/1000
loss: 0.6911903619766235
---------------
Batch 517/1000
loss: 0.69052004814

---------------
Batch 647/1000
loss: 0.6898516416549683
---------------
Batch 648/1000
loss: 0.6901575922966003
---------------
Batch 649/1000
loss: 0.69154953956604
---------------
Batch 650/1000
loss: 0.6924563050270081
---------------
Batch 651/1000
loss: 0.6911908388137817
---------------
Batch 652/1000
loss: 0.6922785639762878
---------------
Batch 653/1000
loss: 0.6906301975250244
---------------
Batch 654/1000
loss: 0.6916425824165344
---------------
Batch 655/1000
loss: 0.6913838982582092
---------------
Batch 656/1000
loss: 0.6917815208435059
---------------
Batch 657/1000
loss: 0.6916688084602356
---------------
Batch 658/1000
loss: 0.6907477378845215
---------------
Batch 659/1000
loss: 0.6910184025764465
---------------
Batch 660/1000
loss: 0.6907271146774292
---------------
Batch 661/1000
loss: 0.6901126503944397
---------------
Batch 662/1000
loss: 0.6898486018180847
---------------
Batch 663/1000
loss: 0.6903380751609802
---------------
Batch 664/1000
loss: 0.69028532505

---------------
Batch 794/1000
loss: 0.6915391087532043
---------------
Batch 795/1000
loss: 0.6899888515472412
---------------
Batch 796/1000
loss: 0.6910008788108826
---------------
Batch 797/1000
loss: 0.6918140649795532
---------------
Batch 798/1000
loss: 0.6905990839004517
---------------
Batch 799/1000
loss: 0.6929172277450562
---------------
Batch 800/1000
loss: 0.6923649311065674
---------------
Batch 801/1000
loss: 0.6915923357009888
---------------
Batch 802/1000
loss: 0.6894662380218506
---------------
Batch 803/1000
loss: 0.691598653793335
---------------
Batch 804/1000
loss: 0.6912749409675598
---------------
Batch 805/1000
loss: 0.6912208795547485
---------------
Batch 806/1000
loss: 0.6906481981277466
---------------
Batch 807/1000
loss: 0.6906620860099792
---------------
Batch 808/1000
loss: 0.6898016929626465
---------------
Batch 809/1000
loss: 0.6909388303756714
---------------
Batch 810/1000
loss: 0.6907410621643066
---------------
Batch 811/1000
loss: 0.6924045681

---------------
Batch 941/1000
loss: 0.689129650592804
---------------
Batch 942/1000
loss: 0.6924287676811218
---------------
Batch 943/1000
loss: 0.6896647214889526
---------------
Batch 944/1000
loss: 0.6913031935691833
---------------
Batch 945/1000
loss: 0.6909015774726868
---------------
Batch 946/1000
loss: 0.6922134757041931
---------------
Batch 947/1000
loss: 0.6914724707603455
---------------
Batch 948/1000
loss: 0.6913470029830933
---------------
Batch 949/1000
loss: 0.6916751861572266
---------------
Batch 950/1000
loss: 0.6918250322341919
---------------
Batch 951/1000
loss: 0.6903758645057678
---------------
Batch 952/1000
loss: 0.6901472210884094
---------------
Batch 953/1000
loss: 0.6938400268554688
---------------
Batch 954/1000
loss: 0.6903380751609802
---------------
Batch 955/1000
loss: 0.6913197636604309
---------------
Batch 956/1000
loss: 0.691271185874939
---------------
Batch 957/1000
loss: 0.6913482546806335
---------------
Batch 958/1000
loss: 0.69050109386

# Testing Outputs

In [10]:
try:
    inputs, outputs, paths = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    #batch = 
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

# for tensor in labels:
#     #tensor = labels[0]
#     tensor_max_value = torch.max(tensor)
#     print(tensor_max_value)
#     print(type(tensor_max_value.item()))
    

In [11]:
dat = dataGenerator.dataGenerator(416)
j = 0
outputs_dir = "data/outputs/" + str(datetime.now().date())
os.makedirs(outputs_dir, exist_ok=True)
threshold = 0.05
batch = 3

for locations in paths:
    img = Image.open(locations)
    img = dat.showMaskOnImage(img,outputs[j], threshold)
    img.save(outputs_dir  + "/batch_" + str(batch) + "-" + str(j) + ".png","PNG")
    #img.show()
    j += 1

#Cells
outputs[0]
    
#Cells
locations = "/data/photos/999.png"
loc = locations.replace("/data/photos/", "").replace(".png", "")

print(locations)
print(loc)

In [8]:
model = UNet16_5(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
model.load_state_dict(torch.load(LOAD_LOCATION+'_test-2019-04-23-3'))
torch.save(model, LOAD_LOCATION)

In [3]:
model = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace=True),
    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
input = torch.rand(1, 3, 216, 216)

In [4]:
input.size()

torch.Size([1, 3, 216, 216])

In [5]:
output = model(input)

In [6]:
output.size()

torch.Size([1, 64, 108, 108])

In [4]:
torchvision.models.vgg16(pretrained=True).features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(